In [5]:
# importing important libraries
import os
import sys
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
from docx import Document
import gradio as gr
import requests

In [2]:
load_dotenv(override=True)
openai = OpenAI()
llama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

In [3]:
pushover_user = os.getenv('PUSHOVER_USER')
pushover_token = os.getenv('PUSHOVER_TOKEN')
pushover_url = 'https://api.pushover.net/1/messages.json'

In [6]:
def push(message):
    print(f"Pushover message: {message}")
    payload = {
        'user': pushover_user,
        'token': pushover_token,
        'message': message,
    }
    response = requests.post(pushover_url, data=payload)
    if response.status_code == 200:
        print("Pushover message sent successfully.")
    else:
        print(f"Failed to send Pushover message. Status code: {response.status_code}, Response: {response.text}")


In [8]:
push('Heyyy PK-Pulse is running!')

Pushover message: Heyyy PK-Pulse is running!
Pushover message sent successfully.


In [9]:
# create two tool functions: record_user_details and record_unknown_question

def record_user_details(email, name='Not Provided', phone='Not Provided', notes='Not Provided'):
    """
    Records user details and returns a confirmation message.
    """
    # Here you can implement the logic to store the user details in a database or file
    push(f"User Details Recorded: Email: {email}, Name: {name}, Phone: {phone}, Notes: {notes}")
    return f"User details recorded for {name} with email {email}: OKAY"

def record_unknown_question(question):
    """ Records an unknown question and returns a confirmation message.
    """
    # Here you can implement the logic to store the unknown question in a database or file
    push(f"Unknown Question Recorded: {question}")
    return f"Unknown question recorded: {question}: OKAY"

In [12]:
# creating json tool for llm
tools_dict = {
"tool_record_user_details_json":{
    "name" : "record_user_details",
    "description" : "Use this tool to record user details like email, name, phone, and notes, if user wants to get in touch",
    "parameters":{
        "type":"object",
        "properties":{
            "email":{
                "type":"string",
                "description":"The email address of the user"
            },
            "name":{
                "type":"string",
                "description":"The name of the user, if provided"
            },
            "phone":{
                "type":"string",
                "description":"The phone number of the user, if provided"
            },
            "notes":{
                "type":"string",
                "description":"Any additional notes or information about the user"
            }
        },
        "required":["email"],
        "additionalProperties":False
    }
},
'tool_record_unknown_question_json':{
    "name": "record_unknown_question",
    "description": "Use this tool to record an unknown question that the user has asked and you are not able to answer",
    "parameters":{
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The unknown question that the user has asked"
            }
        },
        "required": ["question"],
        "additionalProperties": False
    }
}    
}

In [13]:
tools = [{"type": "function", "function": tools_dict["tool_record_user_details_json"]},
         {"type": "function", "function": tools_dict["tool_record_unknown_question_json"]}]

In [38]:
# now the if else logic for the tools (behind the scenes of llm :p )
import json
def tool_call_handler(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        tool_args = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}, Arguments: {tool_args}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**tool_args) if tool else f"Tool {tool_name} not found."
        results.append({"role":"tool", "content":json.dumps(result), "tool_call_id":tool_call.id})
    return results

In [39]:
globals()['record_unknown_question'](("this is a really hard question"))

Pushover message: Unknown Question Recorded: this is a really hard question
Pushover message sent successfully.


'Unknown question recorded: this is a really hard question: OKAY'

In [40]:
about_me_folder_path = './about_me'
# above folder contains the about me files
about_me_files = os.listdir(about_me_folder_path)
# about_me_files = [file for file in about_me_files if file.endswith('.pdf') or file.endswith('.docx')]
# above line filters the files to only include pdf and docx
about_me_files = [file for file in about_me_files if file.endswith('.pdf') or file.endswith('.docx')]
# above line filters the files to only include pdf and docx
about_me_files = [os.path.join(about_me_folder_path, file) for file in about_me_files]
# above line joins the folder path with the file name to get the full path of the file  
def read_pdf(file_path):
    reader = PdfReader(file_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text

def read_docx(file_path):
    doc = Document(file_path)
    text = ''
    for para in doc.paragraphs:
        text += para.text + '\n'
    return text

def read_about_me_files():
    about_me_text = ''
    for file in about_me_files:
        if file.endswith('.pdf'):
            about_me_text += read_pdf(file) + '\n'
        elif file.endswith('.docx'):
            about_me_text += read_docx(file) + '\n'
    contact_info_file = os.path.join(about_me_folder_path, 'contact_info.txt')
    if os.path.exists(contact_info_file):
        with open(contact_info_file, 'r') as f:
            contact_info = f.read()
            about_me_text += contact_info + '\n'
    return about_me_text

# def generate_response(prompt):
#     about_me_text = read_about_me_files()
#     response = openai.chat.completions.create(
#         model="gpt-3.5-turbo",
#         messages=[
#             {"role": "system", "content": "You are a helpful assistant."},
#             {"role": "user", "content": f"{about_me_text}\n\n{prompt}"}
#         ]
#     )
#     return response.choices[0].message['content']

In [41]:
about_me_text = read_about_me_files()

Ignoring wrong pointing object 6 0 (offset 0)
Ignoring wrong pointing object 8 0 (offset 0)
Ignoring wrong pointing object 12 0 (offset 0)
Ignoring wrong pointing object 14 0 (offset 0)
Ignoring wrong pointing object 16 0 (offset 0)
Ignoring wrong pointing object 19 0 (offset 0)
Ignoring wrong pointing object 21 0 (offset 0)
Ignoring wrong pointing object 23 0 (offset 0)
Ignoring wrong pointing object 25 0 (offset 0)
Ignoring wrong pointing object 27 0 (offset 0)
Ignoring wrong pointing object 30 0 (offset 0)


In [42]:
name = 'Priyanshu Khandelwal '
system_prompt = f"""You are representing  {name}. {name} is a data scientist with over 8.5 years of experience. He loves learning about tech. Currently he is is learning French and focusing on fitness. 
You are answering all the question on {name}'s behalf on {name}'s website. Whatever is the answer, first in very short form, reply it in upper caps, then in detail.
You know all about {name}'s career, background, education, skills and interests. You are responsible to represent {name} on 
the interactions as faithfully as possible. You are not allowed to make up any information about {name}.You are given a summary
seperately about {name} which you can use to answer the questions. You have to be proficient and professional and engaging, as you
may be talking to potential employers, clients, or collaborators.. Be faithful, if you don't know the answer to a question, strictly say that you don't know this information about {name}. Don't misrepresent {name} in any way.
You are not allowed to make up any information about {name}. If you are not sure about the answer, say that you are not sure. If you don't know something surely, then don't share it. ITS VERY IMPORTANT.
"""

In [43]:
system_prompt += f"\n\nHere is the summary about {name}:\n{about_me_text}\n\n"
system_prompt += "You are a helpful assistant. You are answering all the questions as {name}.\n\n"

In [ ]:
use_ollama = True
use_ollama_for_evaluation = False
include_history = True
ollama = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama', ) 

In [ ]:
def chat(message, history=[], use_ollama=use_ollama, include_history=include_history):
    if include_history:
        messages = [{"role":"system", "content": system_prompt},] + history[-2:] + [{"role": "user", "content": message}]
    else:
        messages = [{"role":"system", "content": system_prompt}, {"role": "user", "content": message}]
    print(f"User: {message}")
    print(f"Messages: {messages}", flush=True)
    done = False
    while not done:
        if use_ollama:
            print('Using Ollama model')
            response = ollama.chat.completions.create(model='llama3.2', messages=messages, 
                                                    temperature=0.05, 
                                                    max_tokens=1000,
                                                    top_p=0.9,
                                                    frequency_penalty=1.0,
                                                    presence_penalty=0.5,
                                                    # stop=["."],
                                                    tools=tools,
                                                    )
            reply = response.choices[0].message.content
        else:
            print('Using OpenAI model')
            response = openai.chat.completions.create(model='gpt-4o-mini', messages=messages, tools=tools,)
            reply = response.choices[0].message.content
        history.append({"role": "user", "content": message})

        finish_reasoning = response.choices[0].finish_reason
        print(response)
        print(f"Finish Reasoning: {finish_reasoning}", flush=True)
        if finish_reasoning == 'tool_calls':
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = tool_call_handler(tool_calls)
            messages.append(message)
            messages.extend(results)
        # if ('not sure' in (response.choices[0].message.content).lower() 
        # or 'DON\'T HAVE THIS INFORMATION'.lower() in (response.choices[0].message.content).lower()
        # or 'don\'t know this information'.lower() in (response.choices[0].message.content).lower()):
        #     # use tool to record the unknown question
        #     globals()['record_unknown_question'](response.choices[0].message.content)
        #     done = True

        else:
            done=True
    return reply

In [79]:
print(chat("hi", use_ollama=use_ollama))

User: hi
Messages: [{'role': 'system', 'content': "You are representing  Priyanshu Khandelwal . Priyanshu Khandelwal  is a data scientist with over 8.5 years of experience. He loves learning about tech. Currently he is is learning French and focusing on fitness. \nYou are answering all the question on Priyanshu Khandelwal 's behalf on Priyanshu Khandelwal 's website. Whatever is the answer, first in very short form, reply it in upper caps, then in detail.\nYou know all about Priyanshu Khandelwal 's career, background, education, skills and interests. You are responsible to represent Priyanshu Khandelwal  on \nthe interactions as faithfully as possible. You are not allowed to make up any information about Priyanshu Khandelwal .You are given a summary\nseperately about Priyanshu Khandelwal  which you can use to answer the questions. You have to be proficient and professional and engaging, as you\nmay be talking to potential employers, clients, or collaborators.. Be faithful, if you don't

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


User: Hi
Messages: [{'role': 'system', 'content': "You are representing  Priyanshu Khandelwal . Priyanshu Khandelwal  is a data scientist with over 8.5 years of experience. He loves learning about tech. Currently he is is learning French and focusing on fitness. \nYou are answering all the question on Priyanshu Khandelwal 's behalf on Priyanshu Khandelwal 's website. Whatever is the answer, first in very short form, reply it in upper caps, then in detail.\nYou know all about Priyanshu Khandelwal 's career, background, education, skills and interests. You are responsible to represent Priyanshu Khandelwal  on \nthe interactions as faithfully as possible. You are not allowed to make up any information about Priyanshu Khandelwal .You are given a summary\nseperately about Priyanshu Khandelwal  which you can use to answer the questions. You have to be proficient and professional and engaging, as you\nmay be talking to potential employers, clients, or collaborators.. Be faithful, if you don't